In [2]:
#os stuff
import os
import sys
import h5py as h5
import re
import json

#argument parsing
import argparse

#timing
import time

#numpy
import numpy as np

#tensorflow
sys.path.append("/global/homes/t/tkurth/python/tfzoo/tensorflow_mkl_hdf5_mpi_cw")
import tensorflow as tf
import tensorflow.contrib.keras as tfk

#slurm helpers
sys.path.append("../")
import slurm_tf_helper.setup_clusters as sc

#housekeeping
import networks.binary_classifier_tf as bc

# Useful Functions

In [ ]:
def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", type=str, help="specify a config file in json format")
    parser.add_argument("--num_tasks", type=int, default=1, help="specify the number of tasks")
    parser.add_argument("--num_ps", type=int, default=0, help="specify the number of parameters servers")
    pargs = parser.parse_args()
    
    #load the json:
    with open(pargs.config,"r") as f:
        args = json.load(f)
    
    #set the rest
    args['num_tasks'] = pargs.num_tasks
    args['num_ps'] = pargs.num_ps
    
    #modify the activations
    if args['conv_params']['activation'] == 'ReLU':
        args['conv_params']['activation'] = tf.nn.relu
    else:
        raise ValueError('Only ReLU is supported as activation')
        
    #modify the initializers
    if args['conv_params']['initializer'] == 'HE':
        args['conv_params']['initializer'] = tfk.initializers.he_normal()
    else:
        raise ValueError('Only ReLU is supported as initializer')
    
    #now, see if all the paths are there
    args['logpath'] = args['outputpath']+'/logs'
    args['modelpath'] = args['outputpath']+'/models'
    
    if not os.path.isdir(args['logpath']):
        print("Creating log directory ",args['logpath'])
        os.makedirs(args['logpath'])
    if not os.path.isdir(args['modelpath']):
        print("Creating model directory ",args['modelpath'])
        os.makedirs(args['modelpath'])
    if not os.path.isdir(args['inputpath']):
        raise ValueError("Please specify a valid path with input files in hdf5 format")
    
    return args

In [3]:
def train_loop(sess,train_step,args,trainset,validationset):
    
    #counter stuff
    trainset.reset()
    validationset.reset()
    
    #restore weights belonging to graph
    epochs_completed = 0
    if not args['restart']:
        last_model = tf.train.latest_checkpoint(modelpath)
        print("Restoring model %s.",last_model)
        model_saver.restore(sess,last_model)
        epochs_completed = int(re.match(r'^.*?\_epoch\_(.*?)\.ckpt.*?$',last_model).groups()[0])
        trainset._epochs_completed = epochs_completed
    
    #losses
    train_loss=0.
    train_batches=0
    total_batches=0
    train_time=0
    
    #do training
    while (epochs_completed < args['num_epochs']) and not sess.should_stop():
        
        #increment total batch counter
        total_batches+=1
        
        #get next batch
        images,labels,normweights,_ = trainset.next_batch(args['train_batch_size_per_node'])
        #set weights to zero
        normweights[:] = 1.
        
        #update weights
        start_time = time.time()
        if args['create_summary']:
            _, summary, tmp_loss, pred = sess.run([train_step, train_summary, loss_fn, pred_fn],
                                                  feed_dict={variables['images_']: images, 
                                                  variables['labels_']: labels, 
                                                  variables['weights_']: normweights, 
                                                  variables['keep_prob_']: args['dropout_p']})
        else:
            _, tmp_loss, pred = sess.run([train_step, loss_fn, pred_fn],
                                        feed_dict={variables['images_']: images, 
                                        variables['labels_']: labels, 
                                        variables['weights_']: normweights, 
                                        variables['keep_prob_']: args['dropout_p']})
        end_time = time.time()
        train_time += end_time-start_time
        
        #add to summary
        if args['create_summary']:
            with tf.device(args['device']):
                train_writer.add_summary(summary, total_batches)
        
        #increment train loss and batch number
        train_loss += tmp_loss
        train_batches += 1
        
        #determine if we give a short update:
        if train_batches%args['display_interval']==0:
            print("REPORT epoch %d.%d, average training loss %g (%.3f sec/batch)"%(epochs_completed, train_batches,
                                                                                train_loss/float(train_batches),
                                                                                train_time/float(train_batches)))
        
        #check if epoch is done
        if trainset._epochs_completed>epochs_completed:
            epochs_completed=trainset._epochs_completed
            print("COMPLETED epoch %d, average training loss %g (%.3f sec/batch)"%(epochs_completed, 
                                                                                 train_loss/float(train_batches),
                                                                                 train_time/float(train_batches)))
            train_loss=0.
            train_batches=0
            train_time=0
            
            #compute validation loss:
            #reset variables
            validation_loss=0.
            validation_batches=0
            sess.run(init_local_op)
            
            #iterate over batches
            while True:
                #get next batch
                images,labels,normweights,weights = validationset.next_batch(args['validation_batch_size_per_node'])
                #set weights to 1:
                normweights[:] = 1.
                weights[:] = 1.
                
                #compute loss
                if args['create_summary']:
                    summary, tmp_loss=sess.run([validation_summary,loss_fn],
                                                feed_dict={variables['images_']: images, 
                                                            variables['labels_']: labels, 
                                                            variables['weights_']: normweights, 
                                                            variables['keep_prob_']: 1.0})
                else:
                    tmp_loss=sess.run([loss_fn],
                                    feed_dict={variables['images_']: images, 
                                                variables['labels_']: labels, 
                                                variables['weights_']: normweights, 
                                                variables['keep_prob_']: 1.0})
                
                #add loss
                validation_loss += tmp_loss[0]
                validation_batches += 1
                
                #update accuracy
                sess.run(accuracy_fn[1],feed_dict={variables['images_']: images, 
                                                    variables['labels_']: labels, 
                                                    variables['weights_']: weights, 
                                                    variables['keep_prob_']: 1.0})
                
                #update auc
                sess.run(auc_fn[1],feed_dict={variables['images_']: images, 
                                              variables['labels_']: labels, 
                                              variables['weights_']: weights, 
                                              variables['keep_prob_']: 1.0})
                                
                #check if full pass done
                if validationset._epochs_completed>0:
                    validationset.reset()
                    break
                    
            print("COMPLETED epoch %d, average validation loss %g"%(epochs_completed, validation_loss/float(validation_batches)))
            validation_accuracy = sess.run(accuracy_fn[0])
            print("COMPLETED epoch %d, average validation accu %g"%(epochs_completed, validation_accuracy))
            validation_auc = sess.run(auc_fn[0])
            print("COMPLETED epoch %d, average validation auc %g"%(epochs_completed, validation_auc))
            
            # Save the variables to disk.
            if epochs_completed%args['save_interval']==0:
                model_save_path = model_saver.save(sess, args['modelpath']+'/hep_classifier_tfmodel_epoch_'+str(epochs_completed)+'.ckpt')
                print 'Model saved in file: %s'%model_save_path

# Parse Parameters

In [ ]:
args = parse_arguments()

# Multi-Node Stuff

In [6]:
#decide who will be worker and who will be parameters server
if args['num_tasks'] > 1:
    args['cluster'], args['server'], args['task_index'], args['num_tasks'], args['node_type'] = sc.setup_slurm_cluster(num_ps=args['num_ps'])
    if args['node_type'] == "ps":
        args['server'].join()
    elif args['node_type'] == "worker":
        args['is_chief']=(args['task_index'] == 0)
    args['target']=args['server'].target
    if args['num_hot_spares']>=args['num_tasks']-args['num_ps']:
        raise ValueError("The number of hot spares has be be smaller than the number of workers.")
else:
    args['cluster']=None
    args['server']=None
    args['task_index']=0
    args['node_type']='worker'
    args['is_chief']=True
    args['target']=''
    args['hot_spares']=0
    
#general stuff
args["train_batch_size_per_node"]=args["train_batch_size"]/(args["num_tasks"]-args["num_ps"])
args["validation_batch_size_per_node"]=args["validation_batch_size"]/(args["num_tasks"]-args["num_ps"])

# On-Node Stuff

In [ ]:
if (args['node_type'] == 'worker'):
    #common stuff
    os.environ["KMP_BLOCKTIME"] = "1"
    os.environ["KMP_SETTINGS"] = "1"
    os.environ["KMP_AFFINITY"]= "granularity=fine,compact,1,0"

    #arch-specific stuff
    if args['arch']=='hsw':
        num_inter_threads = 2
        num_intra_threads = 16
    elif args['arch']=='knl':
        num_inter_threads = 2
        num_intra_threads = 66
    elif args['arch']=='k80':
        #use default settings
        p = tf.ConfigProto()
        num_inter_threads = int(getattr(p,'INTER_OP_PARALLELISM_THREADS_FIELD_NUMBER'))
        num_intra_threads = int(getattr(p,'INTRA_OP_PARALLELISM_THREADS_FIELD_NUMBER'))
    else:
        raise ValueError('Please specify a valid architecture with arch (allowed values: hsw, knl.)')

    #set the rest
    os.environ['OMP_NUM_THREADS'] = str(num_intra_threads)
    sess_config=tf.ConfigProto(inter_op_parallelism_threads=num_inter_threads,
                               intra_op_parallelism_threads=num_intra_threads,
                               allow_soft_placement=True)

    print("Rank",args['task_index'],": using ",num_inter_threads,"-way task parallelism with ",num_intra_threads,"-way data parallelism.")

## Build Network and Functions

In [ ]:
if args['node_type'] == 'worker':
    print("Rank",args["task_index"],":","Building model")
    args['device'] = tf.train.replica_device_setter(worker_device="/job:worker/task:%d" % args['task_index'],cluster=args['cluster'])
    with tf.device(args['device']):
        variables, network = bc.build_cnn_model(args)
        variables, pred_fn, loss_fn, accuracy_fn, auc_fn = bc.build_functions(args,variables,network)
        tf.add_to_collection('pred_fn', pred_fn)
        tf.add_to_collection('loss_fn', loss_fn)
        print "Rank",args["task_index"],":",variables
        print "Rank",args["task_index"],":",network

## Setup Iterators

In [ ]:
if args['node_type'] == 'worker':
    print("Rank",args["task_index"],":","Setting up iterators")
    
    #training files
    trainfiles = [args['inputpath']+'/'+x for x in os.listdir(args['inputpath']) if 'train' in x and (x.endswith('.h5') or x.endswith('.hdf5'))]
    trainset = bc.DataSet(trainfiles,args['num_tasks'],args['task_index'],split_filelist=True,split_file=False)
    
    #validation files
    validationfiles = [args['inputpath']+'/'+x for x in os.listdir(args['inputpath']) if 'val' in x and (x.endswith('.h5') or x.endswith('.hdf5'))]
    validationset = bc.DataSet(validationfiles,args['num_tasks'],args['task_index'],split_filelist=True,split_file=False)

# Train Model

In [ ]:
#determining which model to load:
metafilelist = [args['modelpath']+'/'+x for x in os.listdir(args['modelpath']) if x.endswith('.meta')]
if not metafilelist:
    #no model found, restart from scratch
    args['restart']=True

In [ ]:
#initialize session
if (args['node_type'] == 'worker'):
    
    with tf.device(args['device']):
        
        #a hook that will stop training at
        hooks=[tf.train.StopAtStepHook(last_step=1000000)]
        
        #global step that either gets updated after any node processes a batch (async) or when all nodes process a batch for a given iteration (sync)
        global_step = tf.train.get_or_create_global_step()     
        opt = tf.train.AdamOptimizer(args['learning_rate'])
        if args['mode'] == "sync":
            #if syncm we make a data structure that will aggregate the gradients form all tasks (one task per node in thsi case)
            opt = tf.train.SyncReplicasOptimizer(opt, replicas_to_aggregate=args['num_tasks'], total_num_replicas=args['num_tasks']-args['num_hot_spares'])
        train_step = opt.minimize(loss_fn, global_step=global_step)
        
        if args["mode"] == "sync":
            hooks.append(opt.make_session_run_hook(is_chief=args['is_chief']))
            
        #creating summary
        if args['create_summary']:
            var_summary = []
            for item in variables:
                var_summary.append(tf.summary.histogram(item,variables[item]))
            summary_loss = tf.summary.scalar("loss",loss_fn)
            summary_accuracy = tf.summary.scalar("accuracy",accuracy_fn)
            train_summary = tf.summary.merge([summary_loss]+var_summary)
            validation_summary = tf.summary.merge([summary_loss])
            
        # Add an op to initialize the variables.
        init_global_op = tf.global_variables_initializer()
        init_local_op = tf.local_variables_initializer()
        
        #saver class:
        model_saver = tf.train.Saver()
        
    
    print("Rank",args["task_index"],": starting training")
    with tf.train.MonitoredTrainingSession(config=sess_config, 
                                           is_chief=args["is_chief"],
                                           master=args['target'],
                                           #checkpoint_dir=args['modelpath'],
                                           hooks=hooks) as sess:
    
        #create summaries
        if args['create_summary']:
            with tf.device(args['device']):
                train_writer = tf.summary.FileWriter(args['logpath'], sess.graph)
                var_summary = []
                for item in variables:
                    var_summary.append(tf.summary.histogram(item,variables[item]))
                summary_loss = tf.summary.scalar("loss",loss_fn)
                summary_accuracy = tf.summary.scalar("accuracy",accuracy_fn)
                train_summary = tf.summary.merge([summary_loss]+var_summary)
                validation_summary = tf.summary.merge([summary_loss])
                train_writer = tf.summary.FileWriter(args['logpath'], sess.graph)
    
        #initialize variables
        sess.run([init_global_op, init_local_op])
    
        #do the training loop
        train_loop(sess,train_step,args,trainset,validationset)